### Loading Required Modules

In [1]:
"""
Loading Required Modules
"""
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.converter import TextConverter
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from io import BytesIO
from io import StringIO
import os
import pandas as pd
import numpy as np

## Keyword Search Utility 
from flashtext import KeywordProcessor
import re
import string

from cleanco import cleanco
import re 
import string

import pdf_to_html as pth
import multiprocessing as mp

In [2]:
"""
Define The Paths
"""
RAW_DATA_PATH = "../2. Data/raw"
HTML_PATH = "../2. Data/html"
TXT_PATH = "../2. Data/txt"
PROT_DF_PATH =  "../2. Data/prot_df"

### Helper Functions

In [3]:
def convertPDFToText_String(path):
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr,codec=codec,laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()
        retstr.close()
        return string
    except Exception as e:
        print(traceback.format_exc())
        print(e)
        return "Fail"

In [4]:
def clean_text_func(text):
        text = str(text)
        text = text.lower()
        text = re.sub(r'[^\x00-\x7F]+',' ', text)
        text = text.replace("\n", " ")
        translator = re.compile('[%s]' % re.escape(string.punctuation))
        text = translator.sub(' ', text)
        text = re.sub(' +', ' ', text)
        text = " ".join(text.split())
        text = text.strip()
        return text

class basic_keyword_extract:
    """Basic keyword extractor class using the keyword processor by Flashtext"""
    def __init__(self, keyword_list, case_sensitive=False):
        self.keyword_list = keyword_list
        self.case_sensitive = case_sensitive
        self.keyword_processor = self._initialize_basic_keyword_processor()
   
    def _initialize_basic_keyword_processor(self):
        kp = KeywordProcessor(case_sensitive=self.case_sensitive)
        kp.add_keywords_from_list(self.keyword_list)
        return kp
    def extract_from_text(self, raw_text):
        if self.case_sensitive:
            text = raw_text
        else:
            text = raw_text.lower().strip()
            clean_text = clean_text_func(text)
            
        clean_extracted_keywords_ls = self.keyword_processor.extract_keywords(clean_text, span_info=True)
#         output_ls = list(set([i[0] for i in clean_extracted_keywords_ls]))
        output_ls = [{'keyword':i[0], 'start_idx':i[1], 'end_idx':i[2]} for i in clean_extracted_keywords_ls]
        output_ls = list({i['keyword']:i for i in reversed(output_ls)}.values()) ## Remove dups
        if len(output_ls) == 0:
            return []
        else:
            return output_ls

In [5]:
def clean_company_name(text):
    
    text = cleanco(text).clean_name()
    text = str(text)
    text = text.lower()
    text = text.lower().replace(' mfg. co. ltd.', ' ').replace('group', '').replace(
                ' & co. ltd.', ' ').replace(' co. ltd.', ' ').replace(' ltd.', ' ').replace('ltd', '').replace('corporation', '').replace(
                'limited', ' ').replace(' pvt.', ' ').replace('pvt', '').replace('company', ' ').replace('india', '').replace('private', '').strip()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

In [6]:
def clean_special_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(r'(?<=\b[a-z]) (?=[a-z]\b)', '', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

In [7]:
def clean_ner_text(text):
    text = str(text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    return text

### Experience

#### Company Extraction

In [8]:
## Company Names
file = "../2. Data/indian_companies_list_v5.txt"
file = open(file, "r", encoding='utf-8')

## Create Company List
COMPANIES_LIST = [line.strip().lower() for line in file]

In [9]:
## Initialize Keyword Processor:
COMPANY_KEYWORDS_SEARCH = basic_keyword_extract(COMPANIES_LIST, case_sensitive=False)

In [10]:
## Function to extract company names
def get_company_names(text):
    text = clean_special_text(text)
    company_ls = COMPANY_KEYWORDS_SEARCH.extract_from_text(text)
    return company_ls

In [11]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company = get_company_names(text)
    print("Companies: ", company, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Companies:  [{'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Companies:  [{'keyword': 'arthur andersen', 'start_idx': 4766, 'end_idx': 4781}, {'keyword': 'bnp paribas', 'start_idx': 218, 'end_idx': 229}, {'keyword': 'ankur seeds', 'start_idx': 4307, 'end_idx': 4318}, {'keyword': 'icici bank', 'start_idx': 230, 'end_idx': 240}, {'keyword': 'religare finvest', 'start_idx': 245, 'end_idx': 261}]
---------------------------------------------- 
File

Companies:  []
---------------------------------------------- 
FileName :  ANSHUL AGARWAL- CV.pdf
Companies:  [{'keyword': 'ernst young', 'start_idx': 2032, 'end_idx': 2043}, {'keyword': 'karvy stock broking limited', 'start_idx': 1574, 'end_idx': 1601}, {'keyword': 'hexaware technologies', 'start_idx': 575, 'end_idx': 596}]
---------------------------------------------- 
FileName :  ANSHUL KUMAR GUPTA- CV (Rinalytics Advisors).pdf
Companies:  [{'keyword': 'uninstall io', 'start_idx': 271, 'end_idx': 283}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Companies:  [{'keyword': 'aon hewitt', 'start_idx': 3880, 'end_idx': 3890}, {'keyword': 'bank muscat', 'start_idx': 861, 'end_idx': 872}, {'keyword': 'hdfc bank', 'start_idx': 1580, 'end_idx': 1589}, {'keyword': 'network 18', 'start_idx': 1519, 'end_idx': 1529}, {'keyword': 'tata motors', 'start_idx': 2083, 'end_idx': 2094}]
---------------------------------------------- 
FileName :  Anshuman Mish

Companies:  [{'keyword': 'torrent pharmaceuticals', 'start_idx': 720, 'end_idx': 743}, {'keyword': 'ajanta pharma', 'start_idx': 1957, 'end_idx': 1970}, {'keyword': 'piramal critical care', 'start_idx': 1805, 'end_idx': 1826}]
---------------------------------------------- 
FileName :  Ashwin N- CV.pdf
Companies:  [{'keyword': 'hsbc', 'start_idx': 1764, 'end_idx': 1768}]
---------------------------------------------- 
FileName :  AVIDHA CHAKRABARTI- CV.pdf
Companies:  [{'keyword': 'icici bank', 'start_idx': 3240, 'end_idx': 3250}, {'keyword': 'morgan stanley', 'start_idx': 219, 'end_idx': 233}]
---------------------------------------------- 
FileName :  Avinash Shekhar- CV.pdf
Companies:  [{'keyword': 'bharti airtel limited', 'start_idx': 526, 'end_idx': 547}, {'keyword': 'bharti enterprises', 'start_idx': 4373, 'end_idx': 4391}, {'keyword': 'future group', 'start_idx': 2457, 'end_idx': 2469}, {'keyword': 'altais advisors private limited', 'start_idx': 3373, 'end_idx': 3404}, {'keyword

Companies:  []
---------------------------------------------- 
FileName :  Dilip Menda- CV (Ishwa).pdf
Companies:  [{'keyword': 'reliance retail', 'start_idx': 6674, 'end_idx': 6689}, {'keyword': 'johnson johnson', 'start_idx': 30, 'end_idx': 45}]
---------------------------------------------- 
FileName :  Dinesh Chhabra- CV.pdf
Companies:  [{'keyword': 'eicher limited', 'start_idx': 1846, 'end_idx': 1860}, {'keyword': 'force motors', 'start_idx': 1530, 'end_idx': 1542}, {'keyword': 'dinesh co', 'start_idx': 3077, 'end_idx': 3086}, {'keyword': 'usha international', 'start_idx': 42, 'end_idx': 60}, {'keyword': 'thirdware solutions', 'start_idx': 1700, 'end_idx': 1719}]
---------------------------------------------- 
FileName :  DIPESH MOHILE- CV.pdf
Companies:  [{'keyword': 'cisco systems', 'start_idx': 1807, 'end_idx': 1820}, {'keyword': 'tonse telecom', 'start_idx': 1556, 'end_idx': 1569}]
---------------------------------------------- 
FileName :  Dipty Jain- CV.pdf
Companies:  [{'ke

Companies:  [{'keyword': 'mcdonald s', 'start_idx': 4066, 'end_idx': 4076}, {'keyword': 'yum restaurants india', 'start_idx': 2567, 'end_idx': 2588}, {'keyword': 'pizza hut', 'start_idx': 1172, 'end_idx': 1181}]
---------------------------------------------- 
FileName :  GIBIN S KOLLAMPARAMBIL- CV.pdf
Companies:  [{'keyword': 'axience consulting', 'start_idx': 1630, 'end_idx': 1648}, {'keyword': 'texmex cuisine', 'start_idx': 688, 'end_idx': 702}, {'keyword': 'venture intelligence', 'start_idx': 333, 'end_idx': 353}]
---------------------------------------------- 
FileName :  Giya Diwaan- CV (ISHWA).pdf
Companies:  [{'keyword': 'idbi bank limited', 'start_idx': 11052, 'end_idx': 11069}, {'keyword': 'moody s analytics', 'start_idx': 9450, 'end_idx': 9467}, {'keyword': 'thomas cook', 'start_idx': 6915, 'end_idx': 6926}, {'keyword': 'canara bank', 'start_idx': 6538, 'end_idx': 6549}, {'keyword': 'essel group', 'start_idx': 5786, 'end_idx': 5797}, {'keyword': 'itzcash card', 'start_idx': 5

...

Companies:  []
---------------------------------------------- 
FileName :  Roopam Upadhyay- CV.pdf
Companies:  [{'keyword': 'major brands india private limited', 'start_idx': 7374, 'end_idx': 7408}, {'keyword': 'wipro infotech', 'start_idx': 3369, 'end_idx': 3383}, {'keyword': 'care kalypto', 'start_idx': 5991, 'end_idx': 6003}, {'keyword': 'kotak securities', 'start_idx': 4999, 'end_idx': 5015}, {'keyword': 'axis bank', 'start_idx': 690, 'end_idx': 699}, {'keyword': 'credit risk management', 'start_idx': 1378, 'end_idx': 1400}, {'keyword': 'future group', 'start_idx': 1200, 'end_idx': 1212}, {'keyword': 'hapag lloyd', 'start_idx': 1075, 'end_idx': 1086}, {'keyword': 'crompton greaves', 'start_idx': 985, 'end_idx': 1001}, {'keyword': 'john deere', 'start_idx': 974, 'end_idx': 984}, {'keyword': 'icici bank', 'start_idx': 920, 'end_idx': 930}, {'keyword': 'mahindra mahindra', 'start_idx': 705, 'end_idx': 722}, {'keyword': 'reliance capital', 'start_idx': 673, 'end_idx': 689}]
-----------

Companies:  [{'keyword': 'ge capital', 'start_idx': 6655, 'end_idx': 6665}, {'keyword': 'genpact india', 'start_idx': 6578, 'end_idx': 6591}, {'keyword': 'tata iq', 'start_idx': 269, 'end_idx': 276}, {'keyword': 'tata steel', 'start_idx': 853, 'end_idx': 863}, {'keyword': 'active marketing', 'start_idx': 478, 'end_idx': 494}, {'keyword': 'tata group', 'start_idx': 422, 'end_idx': 432}]
---------------------------------------------- 
FileName :  Saurabh Gupta- CV.pdf
Companies:  [{'keyword': 'schneider electric', 'start_idx': 13621, 'end_idx': 13639}, {'keyword': 'rockland hospital', 'start_idx': 2123, 'end_idx': 2140}, {'keyword': 'ge healthcare', 'start_idx': 1825, 'end_idx': 1838}, {'keyword': 'tata steel', 'start_idx': 2993, 'end_idx': 3003}, {'keyword': 'fortis healthcare', 'start_idx': 1615, 'end_idx': 1632}, {'keyword': 'sl raheja hospital', 'start_idx': 1596, 'end_idx': 1614}]
---------------------------------------------- 
FileName :  Saurabh SARKAR- CV.pdf
Companies:  [{'keywo

Companies:  [{'keyword': 'state bank of india', 'start_idx': 6742, 'end_idx': 6761}, {'keyword': 'fortis healthcare limited', 'start_idx': 3414, 'end_idx': 3439}, {'keyword': 'fortis healthcare', 'start_idx': 3342, 'end_idx': 3359}, {'keyword': 'shrachi group', 'start_idx': 520, 'end_idx': 533}]
---------------------------------------------- 
FileName :  Sonal Varma- CV.pdf
Companies:  [{'keyword': 'icici bank', 'start_idx': 246, 'end_idx': 256}, {'keyword': 'lehman brothers', 'start_idx': 172, 'end_idx': 187}]
---------------------------------------------- 
FileName :  Sonali Gupta- CV.pdf
Companies:  [{'keyword': 'ericsson india private limited', 'start_idx': 1066, 'end_idx': 1096}]
---------------------------------------------- 
FileName :  Sreedhar Reddy M- CV.pdf
Companies:  []
---------------------------------------------- 
FileName :  Srikanth Nadhamuni- CV.pdf
Companies:  [{'keyword': 'hdfc bank', 'start_idx': 35, 'end_idx': 44}, {'keyword': 'sun microsystems', 'start_idx': 226

Companies:  [{'keyword': 'ernst young llp', 'start_idx': 1150, 'end_idx': 1165}, {'keyword': 'kotak investment banking', 'start_idx': 73, 'end_idx': 97}]
---------------------------------------------- 
FileName :  Usha Prabhakar- CV (ISHWA).pdf
Companies:  [{'keyword': 'hotel crown', 'start_idx': 6497, 'end_idx': 6508}, {'keyword': 'the pioneer', 'start_idx': 5569, 'end_idx': 5580}, {'keyword': 'healthcare at home', 'start_idx': 653, 'end_idx': 671}, {'keyword': 'dhrc hospital', 'start_idx': 2659, 'end_idx': 2672}, {'keyword': 'rockland hospital', 'start_idx': 1810, 'end_idx': 1827}, {'keyword': 'king george hospital', 'start_idx': 1735, 'end_idx': 1755}]
---------------------------------------------- 
FileName :  V Venkatasubramanian- CV (ISHWA).pdf
Companies:  [{'keyword': 'apollo hospitals', 'start_idx': 3371, 'end_idx': 3387}, {'keyword': 'aravind eye care', 'start_idx': 8673, 'end_idx': 8689}, {'keyword': 'sankara nethralaya', 'start_idx': 3054, 'end_idx': 3072}, {'keyword': 'kast

Companies:  [{'keyword': 'hinduja global solutions', 'start_idx': 4306, 'end_idx': 4330}, {'keyword': 'hinduja group', 'start_idx': 4040, 'end_idx': 4053}, {'keyword': 'adm services', 'start_idx': 1642, 'end_idx': 1654}, {'keyword': 'gss infotech limited', 'start_idx': 1539, 'end_idx': 1559}, {'keyword': 'mahesh tutorials', 'start_idx': 667, 'end_idx': 683}, {'keyword': 'mt educare', 'start_idx': 645, 'end_idx': 655}]
---------------------------------------------- 
FileName :  Yash Bansal- CV.pdf
Companies:  [{'keyword': 'grant thornton', 'start_idx': 1709, 'end_idx': 1723}, {'keyword': 'mape advisory group', 'start_idx': 106, 'end_idx': 125}]
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Companies:  [{'keyword': 'arabi company wll', 'start_idx': 7442, 'end_idx': 7459}, {'keyword': 'rdc concrete india', 'start_idx': 4195, 'end_idx': 4213}, {'keyword': 'fosroc chemicals india', 'start_idx': 2922, 'end_idx': 2944}, {'keyword': 'ambuja cements', 'start_id

In [12]:
# company_ls = list(set(company_ls) - set(remove_ls))

# with open('../2. Data/indian_companies_list_v6.txt', 'w', encoding='utf-8') as f:
#     for item in company_ls:
#         f.write("%s\n" % item)

#### Designation Extraction

In [11]:
## Company Names
file = "../2. Data/job_titles_list_v2.txt"
file = open(file, "r", encoding='utf-8')

TITLE_LIST = [line.strip().lower() for line in file]

In [12]:
## Initialize Keyword Processor:
TITLE_KEYWORDS_SEARCH = basic_keyword_extract(TITLE_LIST, case_sensitive=False)

In [13]:
def get_job_titles(text):
    text = clean_special_text(text)
    title_ls = TITLE_KEYWORDS_SEARCH.extract_from_text(text)
    return title_ls

In [16]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    designation = get_job_titles(text)
    print("Designation: ", designation, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Designation:  [{'keyword': 'consultant', 'start_idx': 4325, 'end_idx': 4335}, {'keyword': 'credit manager', 'start_idx': 858, 'end_idx': 872}, {'keyword': 'chartered accountant', 'start_idx': 122, 'end_idx': 142}]
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
Designation:  [{'keyword': 'director', 'start_idx': 16, 'end_idx': 24}, {'keyword': 'senior software developer', 'start_idx': 5256, 'end_idx': 5281}, {'keyword': 'sales marketing', 'start_idx': 4829, 'end_idx': 4844}, {'keyword': '

Designation:  [{'keyword': 'operations analyst', 'start_idx': 7618, 'end_idx': 7636}, {'keyword': 'management trainee', 'start_idx': 7568, 'end_idx': 7586}, {'keyword': 'senior analyst', 'start_idx': 5583, 'end_idx': 5597}, {'keyword': 'securities analyst', 'start_idx': 6050, 'end_idx': 6068}, {'keyword': 'associate', 'start_idx': 4561, 'end_idx': 4570}, {'keyword': 'operator', 'start_idx': 4153, 'end_idx': 4161}, {'keyword': 'cio', 'start_idx': 2352, 'end_idx': 2355}, {'keyword': 'portfolio manager', 'start_idx': 2183, 'end_idx': 2200}, {'keyword': 'founder', 'start_idx': 1806, 'end_idx': 1813}, {'keyword': 'chairman', 'start_idx': 1793, 'end_idx': 1801}, {'keyword': 'senior vice president', 'start_idx': 119, 'end_idx': 140}]
---------------------------------------------- 
FileName :  Amit Singh- CV.pdf
Designation:  [{'keyword': 'legal counsel', 'start_idx': 39, 'end_idx': 52}, {'keyword': 'legal officer', 'start_idx': 3998, 'end_idx': 4011}, {'keyword': 'manager legal', 'start_idx':

Designation:  [{'keyword': 'blog writer', 'start_idx': 1817, 'end_idx': 1828}, {'keyword': 'research intern', 'start_idx': 1256, 'end_idx': 1271}, {'keyword': 'science intern', 'start_idx': 1128, 'end_idx': 1142}, {'keyword': 'data scientist', 'start_idx': 256, 'end_idx': 270}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Designation:  [{'keyword': 'hr business partner', 'start_idx': 5203, 'end_idx': 5222}, {'keyword': 'process owner', 'start_idx': 3793, 'end_idx': 3806}, {'keyword': 'project manager', 'start_idx': 3777, 'end_idx': 3792}, {'keyword': 'graduate engineer', 'start_idx': 1907, 'end_idx': 1924}, {'keyword': 'assistant manager', 'start_idx': 1886, 'end_idx': 1903}, {'keyword': 'hr manager', 'start_idx': 1739, 'end_idx': 1749}, {'keyword': 'senior consultant', 'start_idx': 1721, 'end_idx': 1738}, {'keyword': 'vice president', 'start_idx': 1395, 'end_idx': 1409}, {'keyword': 'human resources professional', 'start_idx': 275, 'end_idx':

Designation:  [{'keyword': 'sr manager', 'start_idx': 7923, 'end_idx': 7933}, {'keyword': 'senior manager', 'start_idx': 7870, 'end_idx': 7884}, {'keyword': 'management trainee', 'start_idx': 7467, 'end_idx': 7485}, {'keyword': 'hr business partner', 'start_idx': 4483, 'end_idx': 4502}, {'keyword': 'sales operations', 'start_idx': 6613, 'end_idx': 6629}, {'keyword': 'regional hr manager', 'start_idx': 1487, 'end_idx': 1506}, {'keyword': 'human resources manager', 'start_idx': 5962, 'end_idx': 5985}, {'keyword': 'practice managers', 'start_idx': 5513, 'end_idx': 5530}, {'keyword': 'process owner', 'start_idx': 4455, 'end_idx': 4468}, {'keyword': 'director', 'start_idx': 2201, 'end_idx': 2209}, {'keyword': 'research manager', 'start_idx': 4147, 'end_idx': 4163}, {'keyword': 'project leader', 'start_idx': 3204, 'end_idx': 3218}, {'keyword': 'executive director', 'start_idx': 3130, 'end_idx': 3148}, {'keyword': 'associate director', 'start_idx': 1259, 'end_idx': 1277}, {'keyword': 'ceo', '

Designation:  [{'keyword': 'advisor consultant', 'start_idx': 4019, 'end_idx': 4037}, {'keyword': 'co founder', 'start_idx': 564, 'end_idx': 574}, {'keyword': 'vp', 'start_idx': 3412, 'end_idx': 3414}, {'keyword': 'managing director', 'start_idx': 2705, 'end_idx': 2722}, {'keyword': 'director', 'start_idx': 36, 'end_idx': 44}, {'keyword': 'vice president', 'start_idx': 1118, 'end_idx': 1132}, {'keyword': 'senior product manager', 'start_idx': 935, 'end_idx': 957}, {'keyword': 'owner', 'start_idx': 871, 'end_idx': 876}, {'keyword': 'avp', 'start_idx': 775, 'end_idx': 778}, {'keyword': 'product development', 'start_idx': 755, 'end_idx': 774}, {'keyword': 'consultant', 'start_idx': 506, 'end_idx': 516}]
---------------------------------------------- 
FileName :  BARNALI ROYCHOWDHURY- CV.pdf
Designation:  [{'keyword': 'management trainee', 'start_idx': 6376, 'end_idx': 6394}, {'keyword': 'ceo', 'start_idx': 743, 'end_idx': 746}, {'keyword': 'recruitment specialist', 'start_idx': 2511, 'end

...

Designation:  [{'keyword': 'owner', 'start_idx': 1422, 'end_idx': 1427}, {'keyword': 'director', 'start_idx': 1240, 'end_idx': 1248}, {'keyword': 'management consultant', 'start_idx': 197, 'end_idx': 218}]
---------------------------------------------- 
FileName :  Shriram Krishnan- CV.pdf
Designation:  [{'keyword': 'finance professional', 'start_idx': 19, 'end_idx': 39}, {'keyword': 'counsel', 'start_idx': 425, 'end_idx': 432}, {'keyword': 'vp', 'start_idx': 253, 'end_idx': 255}, {'keyword': 'cfo', 'start_idx': 123, 'end_idx': 126}, {'keyword': 'consultant', 'start_idx': 59, 'end_idx': 69}]
---------------------------------------------- 
FileName :  SHUBHANGA PRASAD- CV.pdf
Designation:  [{'keyword': 'team lead', 'start_idx': 3853, 'end_idx': 3862}, {'keyword': 'faculty member', 'start_idx': 3020, 'end_idx': 3034}, {'keyword': 'product development', 'start_idx': 2652, 'end_idx': 2671}, {'keyword': 'partner', 'start_idx': 1458, 'end_idx': 1465}, {'keyword': 'digital tech', 'start_idx':

Designation:  [{'keyword': 'certified public accountant', 'start_idx': 2312, 'end_idx': 2339}, {'keyword': 'company secretary', 'start_idx': 1552, 'end_idx': 1569}, {'keyword': 'chartered accountant', 'start_idx': 2075, 'end_idx': 2095}, {'keyword': 'management trainee', 'start_idx': 1956, 'end_idx': 1974}, {'keyword': 'finance executive', 'start_idx': 1884, 'end_idx': 1901}, {'keyword': 'avp', 'start_idx': 1697, 'end_idx': 1700}, {'keyword': 'vp', 'start_idx': 1694, 'end_idx': 1696}, {'keyword': 'financial controller', 'start_idx': 1531, 'end_idx': 1551}, {'keyword': 'cfo', 'start_idx': 1461, 'end_idx': 1464}, {'keyword': 'finance professional', 'start_idx': 139, 'end_idx': 159}]
---------------------------------------------- 
FileName :  Sujan Hajra- CV.pdf
Designation:  [{'keyword': 'executive director', 'start_idx': 12, 'end_idx': 30}, {'keyword': 'director', 'start_idx': 1923, 'end_idx': 1931}, {'keyword': 'vp', 'start_idx': 1867, 'end_idx': 1869}, {'keyword': 'senior vice preside

Designation:  [{'keyword': 'company secretary', 'start_idx': 6503, 'end_idx': 6520}, {'keyword': 'cfo', 'start_idx': 5326, 'end_idx': 5329}, {'keyword': 'manager financial planning', 'start_idx': 873, 'end_idx': 899}, {'keyword': 'business advisor', 'start_idx': 1448, 'end_idx': 1464}, {'keyword': 'financial controller', 'start_idx': 826, 'end_idx': 846}]
---------------------------------------------- 
FileName :  Tejash Mahendra Tanna- CV.pdf
Designation:  [{'keyword': 'manager internal', 'start_idx': 4431, 'end_idx': 4447}, {'keyword': 'cfo', 'start_idx': 369, 'end_idx': 372}, {'keyword': 'chief financial officer', 'start_idx': 345, 'end_idx': 368}, {'keyword': 'chartered accountant', 'start_idx': 22, 'end_idx': 42}]
---------------------------------------------- 
FileName :  Uddeeptabh Mishra- CV (Kelly Services).pdf
Designation:  [{'keyword': 'learning designer', 'start_idx': 5006, 'end_idx': 5023}, {'keyword': 'process owner', 'start_idx': 3262, 'end_idx': 3275}, {'keyword': 'mana

Designation:  [{'keyword': 'regional director', 'start_idx': 22654, 'end_idx': 22671}, {'keyword': 'chief financial officer', 'start_idx': 20774, 'end_idx': 20797}, {'keyword': 'financial controller', 'start_idx': 3563, 'end_idx': 3583}, {'keyword': 'deputy manager', 'start_idx': 3303, 'end_idx': 3317}, {'keyword': 'pricing coordinator', 'start_idx': 19909, 'end_idx': 19928}, {'keyword': 'site controller', 'start_idx': 3010, 'end_idx': 3025}, {'keyword': 'support director', 'start_idx': 15591, 'end_idx': 15607}, {'keyword': 'partner', 'start_idx': 15228, 'end_idx': 15235}, {'keyword': 'business controller', 'start_idx': 2956, 'end_idx': 2975}, {'keyword': 'director', 'start_idx': 539, 'end_idx': 547}, {'keyword': 'sales operations', 'start_idx': 11388, 'end_idx': 11404}, {'keyword': 'cfo', 'start_idx': 565, 'end_idx': 568}, {'keyword': 'site director', 'start_idx': 7709, 'end_idx': 7722}, {'keyword': 'co pilot', 'start_idx': 3851, 'end_idx': 3859}, {'keyword': 'chartered accountant', '

Designation:  [{'keyword': 'tax consultant', 'start_idx': 1957, 'end_idx': 1971}]
---------------------------------------------- 
FileName :  Yatin Joshi- CV.pdf
Designation:  [{'keyword': 'sales manager', 'start_idx': 4946, 'end_idx': 4959}, {'keyword': 'business manager', 'start_idx': 6981, 'end_idx': 6997}, {'keyword': 'technical manager', 'start_idx': 5996, 'end_idx': 6013}, {'keyword': 'sales and marketing manager', 'start_idx': 5705, 'end_idx': 5732}, {'keyword': 'area sales manager', 'start_idx': 5540, 'end_idx': 5558}, {'keyword': 'product development', 'start_idx': 3398, 'end_idx': 3417}, {'keyword': 'national sales manager', 'start_idx': 4172, 'end_idx': 4194}, {'keyword': 'business development manager', 'start_idx': 3723, 'end_idx': 3751}, {'keyword': 'sales marketing', 'start_idx': 96, 'end_idx': 111}, {'keyword': 'area manager', 'start_idx': 2906, 'end_idx': 2918}, {'keyword': 'sales marketing manager', 'start_idx': 2371, 'end_idx': 2394}]
---------------------------------

In [17]:
# title_ls = list(set(title_ls) - set(remove_ls))

# with open('../2. Data/job_titles_list_v2.txt', 'w', encoding='utf-8') as f:
#     for item in title_ls:
#         f.write("%s\n" % item)

#### Linking Company - Designation 

In [14]:
def get_experience(company_output, title_output):
    out_ls = []
    if (len(company_output) > 0) and (len(title_output) > 0):
        for ls_1 in company_output:
            ls = []
            for ls_2 in title_output:
                _dict = {'company_name':ls_1['keyword'], 'title':ls_2['keyword'], 'distance':'', 
                         'company_start_idx':ls_1['start_idx'], 'company_end_idx':ls_1['end_idx']}
                company_end_idx = ls_1['end_idx']
                title_start_idx = ls_2['start_idx']
                _dict['distance'] = np.abs(company_end_idx-title_start_idx)
                ls.append(_dict)
            out_ls.append(min(ls, key=lambda x:x['distance']))
    return out_ls

In [15]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company_output = get_company_names(text)
    title_output = get_job_titles(text)
    experience = get_experience(company_output, title_output)
    print(">> Companies: ", company_output)
    print("\n")
    print(">> Designation: ", title_output)
    print("\n")
    print(">> Experience: ", experience, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> Companies:  [{'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}]


>> Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]


>> Experience:  [{'company_name': 'shah associates', 'title': 'vice president', 'distance': 1988, 

>> Companies:  [{'keyword': 'standard chartered bank', 'start_idx': 5780, 'end_idx': 5803}, {'keyword': 'sbi mutual fund', 'start_idx': 2449, 'end_idx': 2464}, {'keyword': 'state bank of india', 'start_idx': 3752, 'end_idx': 3771}, {'keyword': 'sbi funds management', 'start_idx': 3376, 'end_idx': 3396}, {'keyword': 'mahindra special services group', 'start_idx': 106, 'end_idx': 137}, {'keyword': 'mm group', 'start_idx': 2523, 'end_idx': 2531}]


>> Designation:  [{'keyword': 'manager', 'start_idx': 3891, 'end_idx': 3898}, {'keyword': 'senior manager', 'start_idx': 3834, 'end_idx': 3848}, {'keyword': 'ceo', 'start_idx': 998, 'end_idx': 1001}]


>> Experience:  [{'company_name': 'standard chartered bank', 'title': 'manager', 'distance': 1912, 'company_start_idx': 5780, 'company_end_idx': 5803}, {'company_name': 'sbi mutual fund', 'title': 'senior manager', 'distance': 1370, 'company_start_idx': 2449, 'company_end_idx': 2464}, {'company_name': 'state bank of india', 'title': 'senior manag

>> Companies:  [{'keyword': 'the advertising standards council of india', 'start_idx': 5790, 'end_idx': 5832}, {'keyword': 'colgate palmolive', 'start_idx': 283, 'end_idx': 300}, {'keyword': 'ge money', 'start_idx': 3682, 'end_idx': 3690}, {'keyword': 'hindustan times', 'start_idx': 2861, 'end_idx': 2876}, {'keyword': 'ht media', 'start_idx': 337, 'end_idx': 345}, {'keyword': 'care media', 'start_idx': 1464, 'end_idx': 1474}, {'keyword': 'godrej consumer products', 'start_idx': 231, 'end_idx': 255}]


>> Designation:  [{'keyword': 'management trainee', 'start_idx': 5160, 'end_idx': 5178}, {'keyword': 'manager market research', 'start_idx': 4858, 'end_idx': 4881}, {'keyword': 'product manager', 'start_idx': 4363, 'end_idx': 4378}, {'keyword': 'general manager', 'start_idx': 3979, 'end_idx': 3994}, {'keyword': 'sales marketing', 'start_idx': 3619, 'end_idx': 3634}, {'keyword': 'associate vice president', 'start_idx': 3566, 'end_idx': 3590}, {'keyword': 'marketing lead', 'start_idx': 957,

>> Companies:  [{'keyword': 'abbott labs', 'start_idx': 6751, 'end_idx': 6762}, {'keyword': 'linde gases', 'start_idx': 6735, 'end_idx': 6746}, {'keyword': 'icici bank', 'start_idx': 4742, 'end_idx': 4752}, {'keyword': 'reliance capital ltd', 'start_idx': 6699, 'end_idx': 6719}, {'keyword': 'netambit', 'start_idx': 1323, 'end_idx': 1331}, {'keyword': 'blubirch', 'start_idx': 796, 'end_idx': 804}, {'keyword': 'knoll pharmaceuticals', 'start_idx': 5817, 'end_idx': 5838}, {'keyword': 'icici', 'start_idx': 5379, 'end_idx': 5384}, {'keyword': 'reliance capital', 'start_idx': 2458, 'end_idx': 2474}]


>> Designation:  [{'keyword': 'territory executive', 'start_idx': 5794, 'end_idx': 5813}, {'keyword': 'manager', 'start_idx': 6943, 'end_idx': 6950}, {'keyword': 'branch sales manager', 'start_idx': 4718, 'end_idx': 4738}, {'keyword': 'area sales manager', 'start_idx': 4677, 'end_idx': 4695}, {'keyword': 'regional sales manager', 'start_idx': 3907, 'end_idx': 3929}, {'keyword': 'sales manager',

>> Companies:  [{'keyword': 'tawam hospital', 'start_idx': 2078, 'end_idx': 2092}, {'keyword': 'dubai health authority', 'start_idx': 28596, 'end_idx': 28618}]


>> Designation:  [{'keyword': 'consultant', 'start_idx': 32939, 'end_idx': 32949}, {'keyword': 'registered nurse', 'start_idx': 4617, 'end_idx': 4633}, {'keyword': 'staff nurse', 'start_idx': 32366, 'end_idx': 32377}, {'keyword': 'supervisor', 'start_idx': 27284, 'end_idx': 27294}, {'keyword': 'nurse manager', 'start_idx': 31231, 'end_idx': 31244}, {'keyword': 'charge nurse', 'start_idx': 28812, 'end_idx': 28824}, {'keyword': 'nursing director', 'start_idx': 29024, 'end_idx': 29040}, {'keyword': 'medical practitioners', 'start_idx': 19450, 'end_idx': 19471}, {'keyword': 'supplier quality', 'start_idx': 18409, 'end_idx': 18425}, {'keyword': 'quality coordinator', 'start_idx': 9052, 'end_idx': 9071}, {'keyword': 'safety officer', 'start_idx': 9035, 'end_idx': 9049}, {'keyword': 'infection control coordinator', 'start_idx': 12645

...

>> Companies:  [{'keyword': 'renault nissan', 'start_idx': 8001, 'end_idx': 8015}, {'keyword': 'ford motor', 'start_idx': 3415, 'end_idx': 3425}, {'keyword': 'kaiser permanente', 'start_idx': 6759, 'end_idx': 6776}, {'keyword': 'desert friendly camps private limited', 'start_idx': 5995, 'end_idx': 6032}, {'keyword': 'gabriel india limited', 'start_idx': 58, 'end_idx': 79}, {'keyword': 'tata motors', 'start_idx': 1003, 'end_idx': 1014}]


>> Designation:  [{'keyword': 'managing director', 'start_idx': 7983, 'end_idx': 8000}, {'keyword': 'chairman', 'start_idx': 2805, 'end_idx': 2813}, {'keyword': 'general manager', 'start_idx': 7940, 'end_idx': 7955}, {'keyword': 'corporate controller', 'start_idx': 7301, 'end_idx': 7321}, {'keyword': 'program manager', 'start_idx': 6743, 'end_idx': 6758}, {'keyword': 'supervisor', 'start_idx': 6328, 'end_idx': 6338}, {'keyword': 'director', 'start_idx': 5986, 'end_idx': 5994}, {'keyword': 'coo', 'start_idx': 51, 'end_idx': 54}, {'keyword': 'chief opera

>> Companies:  [{'keyword': 'hdfc life', 'start_idx': 3934, 'end_idx': 3943}, {'keyword': 'computer point limited', 'start_idx': 3615, 'end_idx': 3637}, {'keyword': 'bnp paribas', 'start_idx': 3581, 'end_idx': 3592}, {'keyword': 'srei infrastructure finance limited', 'start_idx': 3545, 'end_idx': 3580}, {'keyword': 'tata steel', 'start_idx': 2285, 'end_idx': 2295}, {'keyword': 'kpmg', 'start_idx': 1849, 'end_idx': 1853}, {'keyword': 'deloitte', 'start_idx': 1025, 'end_idx': 1033}]


>> Designation:  [{'keyword': 'director', 'start_idx': 381, 'end_idx': 389}, {'keyword': 'project management intern', 'start_idx': 3094, 'end_idx': 3119}, {'keyword': 'client representative', 'start_idx': 2979, 'end_idx': 3000}, {'keyword': 'acquisition analyst', 'start_idx': 2608, 'end_idx': 2627}, {'keyword': 'analyst', 'start_idx': 1888, 'end_idx': 1895}, {'keyword': 'senior analyst', 'start_idx': 1076, 'end_idx': 1090}]


>> Experience:  [{'company_name': 'hdfc life', 'title': 'project management intern

>> Companies:  [{'keyword': 'fico', 'start_idx': 7813, 'end_idx': 7817}, {'keyword': 'kajaria ceramics', 'start_idx': 10360, 'end_idx': 10376}, {'keyword': 'varun beverages', 'start_idx': 10284, 'end_idx': 10299}, {'keyword': 'bank of america', 'start_idx': 2022, 'end_idx': 2037}, {'keyword': 'pizza hut', 'start_idx': 9048, 'end_idx': 9057}]


>> Designation:  [{'keyword': 'executive assistant', 'start_idx': 14936, 'end_idx': 14955}, {'keyword': 'engagement manager', 'start_idx': 14825, 'end_idx': 14843}, {'keyword': 'cfo', 'start_idx': 11593, 'end_idx': 11596}, {'keyword': 'project manager', 'start_idx': 1506, 'end_idx': 1521}, {'keyword': 'head of finance', 'start_idx': 12947, 'end_idx': 12962}, {'keyword': 'director', 'start_idx': 2328, 'end_idx': 2336}, {'keyword': 'vice president', 'start_idx': 11574, 'end_idx': 11588}, {'keyword': 'ceo', 'start_idx': 10975, 'end_idx': 10978}, {'keyword': 'people manager', 'start_idx': 10923, 'end_idx': 10937}, {'keyword': 'finance director', 'sta

>> Companies:  [{'keyword': 'tata teleservices', 'start_idx': 212, 'end_idx': 229}, {'keyword': 'jk organisation', 'start_idx': 8393, 'end_idx': 8408}, {'keyword': 'reliance industries limited', 'start_idx': 6615, 'end_idx': 6642}, {'keyword': 'virgin mobile', 'start_idx': 4681, 'end_idx': 4694}, {'keyword': 'tata teleservices limited', 'start_idx': 1423, 'end_idx': 1448}, {'keyword': 'thomas international', 'start_idx': 947, 'end_idx': 967}, {'keyword': 'the chemical company', 'start_idx': 1682, 'end_idx': 1702}, {'keyword': 'jk paper', 'start_idx': 1494, 'end_idx': 1502}]


>> Designation:  [{'keyword': 'chief executive', 'start_idx': 8569, 'end_idx': 8584}, {'keyword': 'sr manager', 'start_idx': 8554, 'end_idx': 8564}, {'keyword': 'partner', 'start_idx': 6951, 'end_idx': 6958}, {'keyword': 'vp', 'start_idx': 6763, 'end_idx': 6765}, {'keyword': 'manager', 'start_idx': 6740, 'end_idx': 6747}, {'keyword': 'project leader', 'start_idx': 6294, 'end_idx': 6308}, {'keyword': 'company manag

>> Companies:  [{'keyword': 'grant thornton', 'start_idx': 1709, 'end_idx': 1723}, {'keyword': 'mape advisory group', 'start_idx': 106, 'end_idx': 125}]


>> Designation:  [{'keyword': 'tax consultant', 'start_idx': 1957, 'end_idx': 1971}, {'keyword': 'internship', 'start_idx': 1324, 'end_idx': 1334}, {'keyword': 'analyst', 'start_idx': 126, 'end_idx': 133}]


>> Experience:  [{'company_name': 'grant thornton', 'title': 'tax consultant', 'distance': 234, 'company_start_idx': 1709, 'company_end_idx': 1723}, {'company_name': 'mape advisory group', 'title': 'analyst', 'distance': 1, 'company_start_idx': 106, 'company_end_idx': 125}]
---------------------------------------------- 
>> FileName :  Yatin Joshi- CV.pdf


>> Companies:  [{'keyword': 'arabi company wll', 'start_idx': 7442, 'end_idx': 7459}, {'keyword': 'rdc concrete india', 'start_idx': 4195, 'end_idx': 4213}, {'keyword': 'fosroc chemicals india', 'start_idx': 2922, 'end_idx': 2944}, {'keyword': 'ambuja cements', 'start_idx': 94

#### Time Period Extraction

In [19]:
import spacy
import datetime
nlp = spacy.load('en_core_web_sm')

In [20]:
def get_period(text):
    text = clean_ner_text(text)
    doc = nlp(text)
    date_ls = []
    if doc.ents: 
        for ent in doc.ents: 
            date_dict = {'period':'', 'start_char':'', 'end_char':''}
            if ent.label_ == "DATE":
                date_dict['period'] = ent.text
                date_dict['start_char'] = ent.start_char
                date_dict['end_char'] = ent.end_char
                if len(DATE_KEYWORDS_SEARCH.extract_from_text(ent.text)) > 0:
                    date_ls.append(date_dict)
    return date_ls

In [81]:
start_year = 1947
end_year = int(datetime.datetime.now().year)
year_ls = [str(i) for i in np.arange(start_year, end_year)]

# DATE_KEYWORDS_LS = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec',
#                     'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november',
#                     'december',
#                     'date', 'present'] + year_ls

DATE_KEYWORDS_LS = year_ls

In [82]:
## Initialize Keyword Processor:
DATE_KEYWORDS_SEARCH = basic_keyword_extract(DATE_KEYWORDS_LS, case_sensitive=False)

In [83]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    tenure_ls = get_period(text)
    print("Time Period: ", tenure_ls, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Time Period:  [{'period': 'Nov 2012', 'start_char': 4198, 'end_char': 4206}, {'period': '2013 Monthly', 'start_char': 4211, 'end_char': 4223}, {'period': 'May 2009', 'start_char': 7605, 'end_char': 7613}, {'period': 'May 2004', 'start_char': 7620, 'end_char': 7628}, {'period': 'May 2004', 'start_char': 8726, 'end_char': 8734}, {'period': '2005', 'start_char': 9081, 'end_char': 9085}]
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
Time Period:  [{'period': 'April 2017- February 2018', 'start_char': 726, 'end_char': 751}, {'period': 'July 2015 April 2017', 'start_char': 1033, 'end_char': 1053}, {'period': 'April 2011 - October 2014', 'start_char': 1975, 'end_char': 2000}, {'period': 'October 2003-March', 'start_char': 3244, 'end_char': 3262}, {'period': 'the years of 2008', 'start_char': 4114, 'end_char': 4131}, {'period': '2009', 'start_char': 4136, 'end_char': 4140}, {'period': 'September 2002 - October 2003', 'start_ch

Time Period:  [{'period': 'May, 2004', 'start_char': 550, 'end_char': 559}, {'period': '2012', 'start_char': 629, 'end_char': 633}, {'period': '2015', 'start_char': 842, 'end_char': 846}, {'period': 'Oct 2015', 'start_char': 941, 'end_char': 949}, {'period': 'May 2011', 'start_char': 1309, 'end_char': 1317}, {'period': 'Oct 2015', 'start_char': 1757, 'end_char': 1765}]
---------------------------------------------- 
FileName :  Aditya Narayan Shetty- CV.pdf
Time Period:  [{'period': 'August 2011', 'start_char': 757, 'end_char': 768}, {'period': 'August 2011', 'start_char': 1027, 'end_char': 1038}, {'period': 'November 2009 to July 2011', 'start_char': 2831, 'end_char': 2857}, {'period': 'May 2012', 'start_char': 4413, 'end_char': 4421}, {'period': '[2002 to 2005', 'start_char': 4778, 'end_char': 4791}, {'period': '2000 to 2002', 'start_char': 4900, 'end_char': 4912}, {'period': 'December 2003', 'start_char': 5117, 'end_char': 5130}, {'period': 'February 27, 1985', 'start_char': 5366, '

Time Period:  [{'period': '2012-13', 'start_char': 5715, 'end_char': 5722}, {'period': '2012-13', 'start_char': 6044, 'end_char': 6051}, {'period': '2012-13', 'start_char': 6231, 'end_char': 6238}, {'period': 'between 2012-13 & 2013-14', 'start_char': 6581, 'end_char': 6606}, {'period': '2012-13', 'start_char': 6666, 'end_char': 6673}, {'period': '2012-13', 'start_char': 6713, 'end_char': 6720}, {'period': '1998', 'start_char': 7807, 'end_char': 7811}, {'period': '1994', 'start_char': 7890, 'end_char': 7894}]
---------------------------------------------- 
FileName :  Anchal Sultania- CV.pdf
Time Period:  [{'period': '2001', 'start_char': 10987, 'end_char': 10991}, {'period': 'March 2010', 'start_char': 11570, 'end_char': 11580}, {'period': 'Sept Dec 2009 July 2005', 'start_char': 11581, 'end_char': 11604}]
---------------------------------------------- 
FileName :  Andrew Stansbie- CV (Hospitale Recruitment).pdf
Time Period:  [{'period': 'August 2007', 'start_char': 328, 'end_char': 3

Time Period:  [{'period': 'August 2013', 'start_char': 514, 'end_char': 525}, {'period': 'December 2014', 'start_char': 828, 'end_char': 841}, {'period': 'May 2016', 'start_char': 1024, 'end_char': 1032}, {'period': 'May / June 2017', 'start_char': 1786, 'end_char': 1801}, {'period': '2008', 'start_char': 2316, 'end_char': 2320}, {'period': '2009', 'start_char': 2325, 'end_char': 2329}]
---------------------------------------------- 
FileName :  ANSHUL KUMAR GUPTA- CV (Rinalytics Advisors).pdf
Time Period:  [{'period': '2012-2016', 'start_char': 80, 'end_char': 89}, {'period': '2011 2009', 'start_char': 121, 'end_char': 130}, {'period': '2013', 'start_char': 1965, 'end_char': 1969}]
---------------------------------------------- 
FileName :  Anshuman Kumar Rath- CV.pdf
Time Period:  [{'period': 'May 2014', 'start_char': 1603, 'end_char': 1611}, {'period': 'April 2015', 'start_char': 1624, 'end_char': 1634}, {'period': 'April 2007', 'start_char': 1691, 'end_char': 1701}, {'period': 'May

Time Period:  [{'period': '2015', 'start_char': 81, 'end_char': 85}, {'period': '2012', 'start_char': 2625, 'end_char': 2629}, {'period': 'July 2009 to March 2012', 'start_char': 5221, 'end_char': 5244}, {'period': '2004', 'start_char': 7468, 'end_char': 7472}, {'period': 'Jan 2008', 'start_char': 7476, 'end_char': 7484}, {'period': 'May 2002', 'start_char': 8477, 'end_char': 8485}, {'period': 'June 1998', 'start_char': 8768, 'end_char': 8777}, {'period': 'May 2002', 'start_char': 8781, 'end_char': 8789}, {'period': '1995', 'start_char': 8837, 'end_char': 8841}, {'period': 'May 1998', 'start_char': 9016, 'end_char': 9024}, {'period': '1992', 'start_char': 9139, 'end_char': 9143}, {'period': '1990', 'start_char': 9203, 'end_char': 9207}, {'period': '25TH August 1974', 'start_char': 9234, 'end_char': 9250}]
---------------------------------------------- 
FileName :  ARNOV KUMAR- CV.pdf
Time Period:  [{'period': 'April 2011', 'start_char': 279, 'end_char': 289}, {'period': 'December 2015'

...

Time Period:  [{'period': 'during1989-1990', 'start_char': 11089, 'end_char': 11104}]
---------------------------------------------- 
FileName :  Srikanth Nadhamuni- CV.pdf
Time Period:  [{'period': '2003', 'start_char': 814, 'end_char': 818}, {'period': 'September 2016 - Present', 'start_char': 1357, 'end_char': 1381}, {'period': 'October 2013', 'start_char': 1452, 'end_char': 1464}, {'period': 'October 2012', 'start_char': 1526, 'end_char': 1538}, {'period': 'July 2009 - September 2012', 'start_char': 1619, 'end_char': 1645}, {'period': 'March 2003 - June 2009', 'start_char': 1939, 'end_char': 1961}, {'period': 'January 2000', 'start_char': 2014, 'end_char': 2026}, {'period': 'February 1996', 'start_char': 2097, 'end_char': 2110}, {'period': 'July 1994 - February 1996', 'start_char': 2186, 'end_char': 2211}, {'period': 'February 1993 - March 1994', 'start_char': 2266, 'end_char': 2292}, {'period': 'January 1989 - February 1993', 'start_char': 2356, 'end_char': 2384}, {'period': '1986

Time Period:  [{'period': 'Dec 2013', 'start_char': 1285, 'end_char': 1293}, {'period': 'Jan 2011 Jan 2013', 'start_char': 2547, 'end_char': 2564}, {'period': '2011', 'start_char': 3105, 'end_char': 3109}, {'period': '2011', 'start_char': 3147, 'end_char': 3151}, {'period': '2012', 'start_char': 3230, 'end_char': 3234}, {'period': '2012', 'start_char': 3319, 'end_char': 3323}, {'period': 'Dec 2010', 'start_char': 3333, 'end_char': 3341}, {'period': 'May 2000', 'start_char': 4854, 'end_char': 4862}, {'period': 'Oct 2004', 'start_char': 4863, 'end_char': 4871}, {'period': '2000', 'start_char': 7278, 'end_char': 7282}, {'period': '1999', 'start_char': 7546, 'end_char': 7550}, {'period': '1997', 'start_char': 9025, 'end_char': 9029}, {'period': '1996', 'start_char': 9198, 'end_char': 9202}, {'period': '2010', 'start_char': 9354, 'end_char': 9358}, {'period': '2009', 'start_char': 9422, 'end_char': 9426}, {'period': '2007', 'start_char': 9534, 'end_char': 9538}, {'period': '2015', 'start_ch

Time Period:  [{'period': '2016 June 2017', 'start_char': 553, 'end_char': 567}, {'period': 'July 2014', 'start_char': 2794, 'end_char': 2803}, {'period': '2016', 'start_char': 3615, 'end_char': 3619}, {'period': '2015', 'start_char': 3802, 'end_char': 3806}, {'period': 'July 2013', 'start_char': 4046, 'end_char': 4055}]
---------------------------------------------- 
FileName :  Usha Prabhakar- CV (ISHWA).pdf
Time Period:  [{'period': '2006 2011', 'start_char': 821, 'end_char': 830}, {'period': '2012', 'start_char': 1302, 'end_char': 1306}, {'period': 'Jan 2006', 'start_char': 2605, 'end_char': 2613}, {'period': 'July-Dec 2012', 'start_char': 2764, 'end_char': 2777}, {'period': 'June 1995', 'start_char': 4614, 'end_char': 4623}, {'period': '2002', 'start_char': 4670, 'end_char': 4674}, {'period': '2002', 'start_char': 4745, 'end_char': 4749}, {'period': '2006', 'start_char': 4789, 'end_char': 4793}, {'period': '2013 M. Phil', 'start_char': 4840, 'end_char': 4852}, {'period': '12th Apr

Time Period:  [{'period': 'October 2007 - December 2009', 'start_char': 597, 'end_char': 625}, {'period': 'March 2007 - October 2007', 'start_char': 706, 'end_char': 731}, {'period': 'June 2005 - March 2007', 'start_char': 789, 'end_char': 811}, {'period': '2000 - 2005', 'start_char': 867, 'end_char': 878}, {'period': '1999 - 2000', 'start_char': 939, 'end_char': 950}, {'period': '1996 - 1999', 'start_char': 1010, 'end_char': 1021}, {'period': '1994 - 1997', 'start_char': 1062, 'end_char': 1073}]
---------------------------------------------- 
FileName :  Vikas Mittal- CV.pdf
Time Period:  [{'period': '2012', 'start_char': 955, 'end_char': 959}, {'period': '2017', 'start_char': 986, 'end_char': 990}, {'period': '2001', 'start_char': 1421, 'end_char': 1425}, {'period': 'the year 2011', 'start_char': 1531, 'end_char': 1544}, {'period': 'February 1977', 'start_char': 1927, 'end_char': 1940}, {'period': '2007', 'start_char': 2254, 'end_char': 2258}, {'period': 'July 2012', 'start_char': 28

Time Period:  [{'period': 'August 2015', 'start_char': 340, 'end_char': 351}, {'period': '2014', 'start_char': 483, 'end_char': 487}, {'period': 'August 2015', 'start_char': 533, 'end_char': 544}, {'period': 'August 2015', 'start_char': 617, 'end_char': 628}, {'period': 'July 2015', 'start_char': 965, 'end_char': 974}, {'period': 'August 2014 to July 2015', 'start_char': 1047, 'end_char': 1071}, {'period': '2011', 'start_char': 1257, 'end_char': 1261}, {'period': 'July 2014', 'start_char': 1267, 'end_char': 1276}, {'period': 'November 2008-Apr 2011', 'start_char': 1998, 'end_char': 2020}, {'period': 'the financial year 2008-09', 'start_char': 2452, 'end_char': 2478}, {'period': 'June 2007', 'start_char': 2664, 'end_char': 2673}, {'period': 'October 2008', 'start_char': 2679, 'end_char': 2691}, {'period': 'May 2005', 'start_char': 3083, 'end_char': 3091}, {'period': 'May 2005', 'start_char': 3203, 'end_char': 3211}, {'period': 'Financial Year 2005-06', 'start_char': 3494, 'end_char': 35

#### Linking Company - Designation - Period

In [84]:
def get_full_experience(company_title_output, date_output):
    out_ls = []
    
    if (len(company_title_output) > 0) and (len(date_output) > 0):
        for ls_1 in company_title_output:
            ls = []
            for ls_2 in date_output:
                _dict = {'company_name':ls_1['company_name'], 'title':ls_1['title'], 'date':ls_2['period'], 'distance_1':ls_1['distance'], 
                         'distance_2':''}
                company_end_idx = ls_1['company_end_idx']
                date_start_idx = ls_2['start_char']
                _dict['distance_2'] = np.abs(company_end_idx-date_start_idx)
                ls.append(_dict)
            out_ls.append(min(ls, key=lambda x:x['distance_2']))
    
    return out_ls

In [85]:
for file in os.listdir(RAW_DATA_PATH):
    if not file.endswith(".pdf"):
        continue
    print(">> FileName : ",file)
    print("\n")
    text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,file))
    company_output = get_company_names(text)
    title_output = get_job_titles(text)
    company_title_output = get_experience(company_output, title_output)
    date_output = get_period(text)
    experience = get_full_experience(company_title_output, date_output)
    print(">> Companies: ", company_output)
    print("\n")
    print(">> Designation: ", title_output)
    print("\n")
    print(">> Experience: ", experience, end="\n---------------------------------------------- \n")

>> FileName :  Abha Doshi- CV.pdf


>> Companies:  [{'keyword': 'shah associates', 'start_idx': 8488, 'end_idx': 8503}, {'keyword': 'fine line circuits limited', 'start_idx': 8409, 'end_idx': 8435}, {'keyword': 'godrej group', 'start_idx': 5486, 'end_idx': 5498}, {'keyword': 'kpmg', 'start_idx': 3839, 'end_idx': 3843}, {'keyword': 'the hershey company', 'start_idx': 5525, 'end_idx': 5544}, {'keyword': 'godrej hershey', 'start_idx': 5363, 'end_idx': 5377}, {'keyword': 'ambit pragma advisors llp', 'start_idx': 1002, 'end_idx': 1027}]


>> Designation:  [{'keyword': 'vice president', 'start_idx': 6515, 'end_idx': 6529}, {'keyword': 'assistant manager', 'start_idx': 5624, 'end_idx': 5641}, {'keyword': 'senior consultant', 'start_idx': 423, 'end_idx': 440}, {'keyword': 'project manager', 'start_idx': 2711, 'end_idx': 2726}, {'keyword': 'finance professional', 'start_idx': 59, 'end_idx': 79}]


>> Experience:  [{'company_name': 'shah associates', 'title': 'vice president', 'date': 'May 2004'

>> Companies:  [{'keyword': 'standard chartered bank', 'start_idx': 5780, 'end_idx': 5803}, {'keyword': 'sbi mutual fund', 'start_idx': 2449, 'end_idx': 2464}, {'keyword': 'state bank of india', 'start_idx': 3752, 'end_idx': 3771}, {'keyword': 'sbi funds management', 'start_idx': 3376, 'end_idx': 3396}, {'keyword': 'mahindra special services group', 'start_idx': 106, 'end_idx': 137}, {'keyword': 'mm group', 'start_idx': 2523, 'end_idx': 2531}]


>> Designation:  [{'keyword': 'manager', 'start_idx': 3891, 'end_idx': 3898}, {'keyword': 'senior manager', 'start_idx': 3834, 'end_idx': 3848}, {'keyword': 'ceo', 'start_idx': 998, 'end_idx': 1001}]


>> Experience:  [{'company_name': 'standard chartered bank', 'title': 'manager', 'date': '2008', 'distance_1': 1912, 'distance_2': 11}, {'company_name': 'sbi mutual fund', 'title': 'senior manager', 'date': '2008-July 2015', 'distance_1': 1370, 'distance_2': 1063}, {'company_name': 'state bank of india', 'title': 'senior manager', 'date': 'Octobe

>> Companies:  [{'keyword': 'toll global logistics', 'start_idx': 1314, 'end_idx': 1335}, {'keyword': 'mahindra mahindra ltd', 'start_idx': 1605, 'end_idx': 1626}, {'keyword': 'tata steel', 'start_idx': 7558, 'end_idx': 7568}, {'keyword': 'allcargo logistics', 'start_idx': 1223, 'end_idx': 1241}, {'keyword': 'india value fund advisors', 'start_idx': 42, 'end_idx': 67}]


>> Designation:  [{'keyword': 'executive assistant', 'start_idx': 18536, 'end_idx': 18555}, {'keyword': 'associate', 'start_idx': 18339, 'end_idx': 18348}, {'keyword': 'financial management', 'start_idx': 1023, 'end_idx': 1043}, {'keyword': 'director', 'start_idx': 15191, 'end_idx': 15199}, {'keyword': 'chief executive', 'start_idx': 15149, 'end_idx': 15164}, {'keyword': 'manufacturing engineer', 'start_idx': 1740, 'end_idx': 1762}, {'keyword': 'manager', 'start_idx': 1507, 'end_idx': 1514}, {'keyword': 'managing director', 'start_idx': 1368, 'end_idx': 1385}, {'keyword': 'vice president', 'start_idx': 9293, 'end_idx':

>> Companies:  [{'keyword': 'abbott labs', 'start_idx': 6751, 'end_idx': 6762}, {'keyword': 'linde gases', 'start_idx': 6735, 'end_idx': 6746}, {'keyword': 'icici bank', 'start_idx': 4742, 'end_idx': 4752}, {'keyword': 'reliance capital ltd', 'start_idx': 6699, 'end_idx': 6719}, {'keyword': 'netambit', 'start_idx': 1323, 'end_idx': 1331}, {'keyword': 'blubirch', 'start_idx': 796, 'end_idx': 804}, {'keyword': 'knoll pharmaceuticals', 'start_idx': 5817, 'end_idx': 5838}, {'keyword': 'icici', 'start_idx': 5379, 'end_idx': 5384}, {'keyword': 'reliance capital', 'start_idx': 2458, 'end_idx': 2474}]


>> Designation:  [{'keyword': 'territory executive', 'start_idx': 5794, 'end_idx': 5813}, {'keyword': 'manager', 'start_idx': 6943, 'end_idx': 6950}, {'keyword': 'branch sales manager', 'start_idx': 4718, 'end_idx': 4738}, {'keyword': 'area sales manager', 'start_idx': 4677, 'end_idx': 4695}, {'keyword': 'regional sales manager', 'start_idx': 3907, 'end_idx': 3929}, {'keyword': 'sales manager',

>> Companies:  [{'keyword': 'tawam hospital', 'start_idx': 2078, 'end_idx': 2092}, {'keyword': 'dubai health authority', 'start_idx': 28596, 'end_idx': 28618}]


>> Designation:  [{'keyword': 'consultant', 'start_idx': 32939, 'end_idx': 32949}, {'keyword': 'registered nurse', 'start_idx': 4617, 'end_idx': 4633}, {'keyword': 'staff nurse', 'start_idx': 32366, 'end_idx': 32377}, {'keyword': 'supervisor', 'start_idx': 27284, 'end_idx': 27294}, {'keyword': 'nurse manager', 'start_idx': 31231, 'end_idx': 31244}, {'keyword': 'charge nurse', 'start_idx': 28812, 'end_idx': 28824}, {'keyword': 'nursing director', 'start_idx': 29024, 'end_idx': 29040}, {'keyword': 'medical practitioners', 'start_idx': 19450, 'end_idx': 19471}, {'keyword': 'supplier quality', 'start_idx': 18409, 'end_idx': 18425}, {'keyword': 'quality coordinator', 'start_idx': 9052, 'end_idx': 9071}, {'keyword': 'safety officer', 'start_idx': 9035, 'end_idx': 9049}, {'keyword': 'infection control coordinator', 'start_idx': 12645

...

>> Companies:  []


>> Designation:  [{'keyword': 'learning designer', 'start_idx': 5006, 'end_idx': 5023}, {'keyword': 'process owner', 'start_idx': 3262, 'end_idx': 3275}, {'keyword': 'management trainee', 'start_idx': 1179, 'end_idx': 1197}, {'keyword': 'manager', 'start_idx': 1162, 'end_idx': 1169}, {'keyword': 'hr manager', 'start_idx': 1145, 'end_idx': 1155}, {'keyword': 'general manager', 'start_idx': 1129, 'end_idx': 1144}, {'keyword': 'chairman', 'start_idx': 747, 'end_idx': 755}]


>> Experience:  []
---------------------------------------------- 
>> FileName :  Ujjal Bhattacharya- CV.pdf


>> Companies:  [{'keyword': 'mjunction services', 'start_idx': 45, 'end_idx': 63}, {'keyword': 'tata steel', 'start_idx': 919, 'end_idx': 929}, {'keyword': 'prime steel', 'start_idx': 418, 'end_idx': 429}]


>> Designation:  [{'keyword': 'vice president', 'start_idx': 19, 'end_idx': 33}, {'keyword': 'manager', 'start_idx': 876, 'end_idx': 883}, {'keyword': 'general manager', 'start_idx': 7

>> Companies:  [{'keyword': 'apollo hospitals', 'start_idx': 3371, 'end_idx': 3387}, {'keyword': 'aravind eye care', 'start_idx': 8673, 'end_idx': 8689}, {'keyword': 'sankara nethralaya', 'start_idx': 3054, 'end_idx': 3072}, {'keyword': 'kasturba hospital', 'start_idx': 6352, 'end_idx': 6369}, {'keyword': 'manipal group', 'start_idx': 5388, 'end_idx': 5401}, {'keyword': 'columbia asia', 'start_idx': 39, 'end_idx': 52}, {'keyword': 'apollo institute', 'start_idx': 3300, 'end_idx': 3316}]


>> Designation:  [{'keyword': 'management trainee', 'start_idx': 10156, 'end_idx': 10174}, {'keyword': 'internship', 'start_idx': 10120, 'end_idx': 10130}, {'keyword': 'internal auditor', 'start_idx': 3626, 'end_idx': 3642}, {'keyword': 'event manager', 'start_idx': 8106, 'end_idx': 8119}, {'keyword': 'manager', 'start_idx': 5456, 'end_idx': 5463}, {'keyword': 'assistant manager', 'start_idx': 5402, 'end_idx': 5419}, {'keyword': 'general manager', 'start_idx': 83, 'end_idx': 98}, {'keyword': 'procurem

>> Companies:  [{'keyword': 'oracle corp', 'start_idx': 54, 'end_idx': 65}, {'keyword': 'hyatt regency', 'start_idx': 19972, 'end_idx': 19985}, {'keyword': 'bristol myers squibb', 'start_idx': 13213, 'end_idx': 13233}, {'keyword': 'infosys', 'start_idx': 404, 'end_idx': 411}, {'keyword': 'infosys limited', 'start_idx': 4144, 'end_idx': 4159}, {'keyword': 'novel technologies', 'start_idx': 5835, 'end_idx': 5853}, {'keyword': 'between technology', 'start_idx': 1168, 'end_idx': 1186}]


>> Designation:  [{'keyword': 'intelligence applications', 'start_idx': 79, 'end_idx': 104}, {'keyword': 'data scientist', 'start_idx': 31, 'end_idx': 45}, {'keyword': 'senior principal', 'start_idx': 14, 'end_idx': 30}, {'keyword': 'staff engineer', 'start_idx': 9005, 'end_idx': 9019}, {'keyword': 'research assistant', 'start_idx': 11493, 'end_idx': 11511}, {'keyword': 'research intern', 'start_idx': 12847, 'end_idx': 12862}, {'keyword': 'software developer', 'start_idx': 12689, 'end_idx': 12707}, {'keywo

>> Companies:  [{'keyword': 'institute of computer', 'start_idx': 8071, 'end_idx': 8092}, {'keyword': 'computer software', 'start_idx': 8010, 'end_idx': 8027}, {'keyword': 'nisa group of companies', 'start_idx': 6022, 'end_idx': 6045}, {'keyword': 'bajaj electricals', 'start_idx': 5413, 'end_idx': 5430}, {'keyword': 'reliance retail', 'start_idx': 4906, 'end_idx': 4921}, {'keyword': 'panache globe solutions', 'start_idx': 193, 'end_idx': 216}]


>> Designation:  [{'keyword': 'manager', 'start_idx': 2578, 'end_idx': 2585}, {'keyword': 'business partner', 'start_idx': 4101, 'end_idx': 4117}, {'keyword': 'regional manager', 'start_idx': 4076, 'end_idx': 4092}, {'keyword': 'partner', 'start_idx': 2623, 'end_idx': 2630}, {'keyword': 'co founder', 'start_idx': 2254, 'end_idx': 2264}, {'keyword': 'director', 'start_idx': 225, 'end_idx': 233}, {'keyword': 'founder', 'start_idx': 217, 'end_idx': 224}]


>> Experience:  [{'company_name': 'institute of computer', 'title': 'business partner', 'dat

>> Companies:  [{'keyword': 'boston consulting group', 'start_idx': 4599, 'end_idx': 4622}, {'keyword': 'tata motors', 'start_idx': 4526, 'end_idx': 4537}, {'keyword': 'capital ltd', 'start_idx': 400, 'end_idx': 411}, {'keyword': 'tata motors finance', 'start_idx': 3960, 'end_idx': 3979}, {'keyword': 'icici bank', 'start_idx': 3056, 'end_idx': 3066}]


>> Designation:  [{'keyword': 'research scholar', 'start_idx': 5730, 'end_idx': 5746}, {'keyword': 'business representative', 'start_idx': 4550, 'end_idx': 4573}, {'keyword': 'visiting professor', 'start_idx': 4059, 'end_idx': 4077}, {'keyword': 'credit manager', 'start_idx': 3824, 'end_idx': 3838}, {'keyword': 'area sales manager', 'start_idx': 2678, 'end_idx': 2696}, {'keyword': 'sales manager', 'start_idx': 3152, 'end_idx': 3165}, {'keyword': 'senior manager', 'start_idx': 1889, 'end_idx': 1903}, {'keyword': 'avp', 'start_idx': 1714, 'end_idx': 1717}, {'keyword': 'organizational development specialist', 'start_idx': 1181, 'end_idx': 1

In [73]:
text = convertPDFToText_String(os.path.join(RAW_DATA_PATH,os.listdir(RAW_DATA_PATH)[2]))

company_output = get_company_names(text)
title_output = get_job_titles(text)

company_title_output = get_experience(company_output, title_output)

date_output = get_period(text)

In [78]:
get_full_experience(company_title_output, date_output)

[{'company_name': 'flipkart',
  'title': 'director',
  'date': 'April 2015 -',
  'distance_1': 44,
  'distance_2': 1203},
 {'company_name': 'interwoven inc',
  'title': 'senior software developer',
  'date': 'July 2005',
  'distance_1': 63,
  'distance_2': 191},
 {'company_name': 'itc limited',
  'title': 'sales marketing',
  'date': '2011 - June 2012',
  'distance_1': 430,
  'distance_2': 338}]

### Skills

In [15]:
file = "../2. Data/skills_clean_list_v2.txt"
file = open(file, "r", encoding='utf-8')

skill = [line.strip().lower() for line in file]

# skill = list(set([clean_text_func(i) for i in skill]))

# skill = list(set(skill) - set([]))

# with open('../2. Data/skills_clean_list_v2.txt', 'w', encoding='utf-8') as f:
#     for item in skill:
#         f.write("%s\n" % item)

In [45]:
## Initialize Keyword Processor:
skills_keywords_search = basic_keyword_extract(skill, case_sensitive=False)

In [46]:
directory = r'C:\Users\admin\OneDrive - True North Managers LLP\Desktop\TrueNorth\1. Projects\1. 3A Team Project\6. CV Extraction\1. Shared\CVs'
for file in os.listdir(directory):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(directory,file))
    skills = list(set(skills_keywords_search.extract_from_text(text)))
    print("Skills: ", skills, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
Skills:  ['promoters', 'mid market', 'investment', 'teasers', 'cxo', 'pay', 'financial planning', 'llp', 'english', 'crm', 'accountants', 'automotive', 'cold chain', 'output', 'general', 'documentation', 'project', 'measures', 'corporate finance', 'order to cash', 'financial statements', 'closure', 'diamonds', 'sports', 'strategy', 'decision making', 'product mix', 'strategic growth', 'dashboard', 'variance analysis', 'synergies', 'deal sourcing', 'academic background', 'cloud', 'gujarati', 'finance', 'acquisition integration', 'functionality', 'fortune 500', 'private equity', 'languages', 'business model innovation', 'gross margin', 'net', 'digital marketing', 'value creation', 'manufacturing', 'studio', 'online presence', 'liability', 'pet food', 'multinational', 'coordinated', 'provisioning', 'ifrs', 'circuits', 'internal controls', 'investments', 'dutch', 'client centric', 'financial advisory', 'due diligence', 'led', 'bookkeeping', 'hindi', 'growth c

Skills:  ['compensation', 'risk consulting', 'birth', 'pay', 'statutory compliances', 'viz', 'english', 'base', 'recruiting', 'regulatory compliance', 'designation', 'sourcing', 'boarding', 'project', 'barriers', 'campus', 'public relations', 'bungalows', 'chemistry', 'mail', 'budgeting', 'asset management', 'employee relations', 'compensation benchmarking', 'investors', 'staff training', 'formulation', 'regulations', 'esic', 'financial services', 'government', 'participation', 'legal requirements', 'cooking', 'benchmarking', 'singing', 'languages', 'marathon', 'banding', 'policies procedures', 'capital markets', 'salary', 'road', 'performance management', 'incentives', 'hr operations', 'man power', 'onboarding', 'revisions', 'web technologies', 'mutual funds', 'research', 'payroll', 'mms', 'hr transformation', 'hindi', 'securities', 'tvc', 'technical advisory', 'reporting', 'profiles', 'interviewing', 'pipeline', 'personnel files', 'fraud', 'intelligence', 'presentations', 'organizati

Skills:  ['compensation', 'retail', 'coil', 'latam', 'reach', 'trailer', 'investment', 'destination services', 'working experience', 'performing', 'fmcg', 'it systems development', 'implementation experience', 'icd', 'cellular manufacturing', 'outsourcing', 'english', 'business travel', 'logistics engineering', 'tax', 'crm', 'base', 'documentation', 'operational excellence', 'project', 'organizational capabilities', 'l d', 'notification', 'strategy', 'transportation management', 'f 15', 'kaizen', 'proposition', 'mail', 'freight forwarding', 'japanese', 'delegation', 'materials management', 'lcl', 'pan', 'mobility', 'www', 'digital strategy', 'transportation', 'information technology', 'east africa', 'sap erp', 'facsimile', 'turnover', 'cab', 'highways', 'change management', 'languages', 'express', 'email', 'business profitability', 'global logistics', 'him operations', 'productivity', 'value creation', 'business solution', 'cost savings', 'union', 'manufacturing', 'sales process', 'pro

Skills:  ['pre opening', 'market development', 'buffet', 'spa', 'iso 22000', 'food safety', 'birth', 'italian cuisine', 'italian', 'preparation', 'organization', 'www', 'towers', 'pools', 'purchasing', 'teaching', 'ilm', 'pastry', 'payroll', 'advanced', 'forecasting', 'bar', 'team development', 'serviced apartments', 'regulations', 'hygiene', 'cafe', 'special events', 'gala dinners', 'world travel', 'cakes', 'suite', 'yahoo', 'cooking', 'events', 'health', 'creole', 'jazz', 'lounge', 'receptions', 'scheduling', 'spaces', 'outlets', 'public health', 'general', 'ministers', 'british', 'calculations', 'operations', 'fits', 'options', 'square', 'email', 'flame', 'documentation', 'suppliers', 'demand', 'vacation planning', 'fish', 'catering', 'principles', 'plus', 'design', 'sandals', 'intermediate', 'seasonal', 'fine dining', 'leadership management', 'further education', 'culinary education', 'molecular gastronomy', 'caf', 'starwood', 'budgeting', 'hospitality management', 'weddings']
----

Skills:  ['security', 'maintenance', 'data analytics', 'logistic regression', 'coordinated', 'badminton', 'app', 'warranty', 'construction', 'tat', 'bosch', 'application', 'public', 'infrastructure', 'data analysis', 'embedded', 'publications', 'packers', 'insurance risk', 'vendors', 'models', 'scripting', 'completion', 'research', 'deals', 'sponsorship', 'damage', 'excess', 'children', 'data acquisition', 'foraging', 'multiprocessing', 'learning', 'gold', 'choreography', 'bronze', 'landscape', 'loan', 'crm', 'data mining', 'optima', 'led', 'spare parts', 'steering', 'nss', 'stock market', 'infosys', 'analytics', 'less', 'paper', 'information gathering', 'excel', 'apt', 'boarding', 'operations', 'dissertation', 'mathematics', 'email', 'originating', 'operational planning', 'project', 'early warning', 'bridge', 'price optimization', 'movers', 'hsr', 'descriptive analysis', 'business solution', 'live', 'strategy', 'computing', 'poker', 'hobbies', 'mobile data', 'python', 'assessment', 'l

...

Skills:  ['aerospace', 'delegation', 'pitch books', 'presentations', 'retail', 'pan', 'ieee', 'flow', 'medals', 'row', 'textiles', 'silver', 'www', 'public', 'pay', 'teaching', 'financial modeling', 'summit', 'agents', 'models', 'buy side', 'fmcg', 'research', 'taekwondo', 'healthcare', 'jerseys', 'dcf', 'reconstruction', 'oil gas', 'viz', 'gold', 'bronze', 'english', 'financial services', 'finance', 'high end', 'roller skating', 'football', 'health', 'private equity', 'led', 'customer', 'pulse', 'record', 'technology', 'analytics', 'excel', 'c i', 'documentation', 'project', 'secondary research', 'mathematics', 'word', 'capital', 'investment banking', 'multiples', 'capital iq', 'arc', 'bloomberg', 'car rental', 'space', 'water', 'store', 'electronics', 'trading', 'sports', 'footprints', 'e commerce', 'showcase', 'sell side', 'manufacturing', 'key metrics', 'pro', 'valuation', 'convenience', 'macro', 'black belt', 'ngo', 'product lifecycle management']
---------------------------------

Skills:  ['general motors', 'business valuation', 'coaching', 'transmission', 'industrial engineering', 'continuous improvement', 'savings', 'automotive', 'general', 'output', 'project', 'business center', 'consolidation', 'manufacturing engineering', 'countermeasures', 'strategy', 'vast', 'llc', 'strategic growth', 'credentials', 'business engineering', 'people development', 'variance analysis', 'synergies', 'business planning', 'motors', 'swot analysis', 'adherence', 'finance', 'spring', 'events', 'benchmarking', 'mentoring', 'marathon', 'act', 'union', 'manufacturing', 'cash flow', 'young professionals', 'physicals', 'coordinated', 'purchasing', 'capital allocation', 'business acumen', 'advanced', 'gears', 'range', 'commitments', 'export', 'customer', 'led', 'communications', 'technology', 'wire', 'operations', 'reporting', 'strategy creation', 'integrative', 'transformational leadership', 'long distance', 'assessment', 'market share', 'financial analysis', 'upgradation', 'presentat

Skills:  ['biscuits', 'paint', 'credit', 'medals', 'milk', 'subsidiaries', 'preparation', 'investment', 'pipes', 'variance analysis', 'strategy formulation', 'synergies', 'deals', 'business planning', 'investments', 'classic', 'restructuring', 'performance measurement', 'polymers', 'gold', 'english', 'financial services', 'finance', 'international business', 'affordable housing', 'bond funds', 'due diligence', 'south pacific', 'personal care', 'ice cream', 'private equity', 'value added', 'led', 'hindi', 'growth capital', 'technology', 'carbon', 'sourcing', 'supply chain', 'operations', 'frozen desserts', 'languages', 'wordpress', 'islands', 'email', 'faucets', 'project', 'measures', 'corporate finance', 'foundation', 'space', 'sports', 'exits', 'consumables', 'proxy', 'e commerce', 'south asia', 'manufacturing', 'emerging markets', 'enterprise', 'cor', 'valuation', 'bengali', 'equities', 'bangladesh', 'tiles', 'housing development']
---------------------------------------------- 
File

Skills:  ['shared services', 'process efficiency', 'compensation', 'pizza', 'statutes', 'reach', 'agenda', 'plants', 'standards based', 'fmcg', 'loans', 'inventory accounting', 'outsourcing', 'english', 'statues', 'grc', 'landscape', 'remediation', 'tax', 'base', 'accountants', 'remittances', 'c a', 'sales automation', 'trade finance', 'severance', 'ratings', 'operational excellence', 'options', 'project', 'unit operations', 'closure', 'financial statements', 'foundation', 'centralization', 'notification', 'strategy', 'automation', 'mail', 'sales tax', 'budgeting', 'mobility', 'www', 'staff training', 'liaison', 'formulation', 'ibm', 'adherence', 'finance', 'team building', 'facsimile', 'procurement', 'change management', 'f a', 'languages', 'email', 'net', 'capital', 'cross country', 'productivity', 'manufacturing', 'business initiatives', 'reference checking', 'interpersonal skills', 'salary', 'instrumental', 'supply chain operations', 'coordinated', 'spanish', 'ceramics', 'kpi', 'bu

Skills:  ['compensation', 'entitlements', 'retail', 'leadership development', 'chemical research', 'birth', 'forming', 'application', 'accelerated growth', 'performing', 'plants', 'sensitivity', 'interventions', 'fuel', 'oil gas', 'csr', 'english', 'higher education', 'designation', 'vendor management', 'vmi', 'sourcing', 'project', 'sales marketing', 'personnel management', 'polyester', 'strategy', 'hris', 'scenario', 'pan', 'construction', 'ppa', 'accountability', 'apps', 'gujarati', 'team building', 'hr strategy', 'cross functional initiatives', 'benchmarking', 'global research', 'less', 'change management', 'tournaments', 'dexterity', 'shaping', 'productivity', 'south asia', 'manufacturing', 'union', 'oracle', 'road', 'value assessment', 'refining', 'performance management', 'r r', 'settlement', 'hr operations', 'coordinated', 'restoration', 'research', 'initiation', 'workshops', 'principle', 'scheme', 'telecom', 'sat', 'commitments', 'due diligence', 'people management', 'hindi', 

### Education

In [16]:
file = "../2. Data/indian_colleges_list_v4.txt"
file = open(file, "r", encoding='utf-8')

college = [line.strip().lower() for line in file]

# college = [i for i in college if len(i) > 1]

# college = list(set([clean_text_func(i) for i in college]))

# college = list(set(college) - set(['graduate school of business',
#                                  'college of commerce',
#                                  'jain institute',
#                                  'institute of business management',
#                                  'medical collegefaculty of management studies',
#                                  'medical college',
#                                  'it solution',
#                                  'nursing college',
#                                  'institute of management studies',
#                                  'law college',
#                                  'college of engineering',
#                                  'command hospital',
#                                  'engineering college',
#                                  'faculty of management',
#                                  'science college',
#                                  'lucknow campus',
#                                  'college of nursing',
#                                  'global technologies',
#                                  'institute of business management',
#                                   'college of management studies', 'faculty of commerce', 
#                                    'institute of management and research',
#                                   'college of arts',
#                                   'institute of child health',
#                                   'kanchi kamakoti childs trust hospital',
#                                   'learning management systems',
#                                   'institute of pharmacy', 'faculty of management studies',
#                                   'national college',
#                                   'govt model', 'vivekananda', 'institute of technology management',
#                                   'computer point', 'arts and commerce',
#                                   'miranda house', 'institute of chemical technology']))

# with open('../2. Data/indian_colleges_list_v4.txt', 'w', encoding='utf-8') as f:
#     for item in college:
#         f.write("%s\n" % item)

In [17]:
## Initialize Keyword Processor:
college_keywords_search = basic_keyword_extract(college, case_sensitive=False)

In [18]:
directory = r'C:\Users\admin\OneDrive - True North Managers LLP\Desktop\TrueNorth\1. Projects\1. 3A Team Project\6. CV Extraction\1. Shared\CVs'
for file in os.listdir(directory):
    if not file.endswith(".pdf"):
        continue
    print("FileName : ",file)
    text = convertPDFToText_String(os.path.join(directory,file))
    college = list(set(college_keywords_search.extract_from_text(text)))
    print("College: ", college, end="\n---------------------------------------------- \n")

FileName :  Abha Doshi- CV.pdf
College:  ['university of mumbai']
---------------------------------------------- 
FileName :  Abhida Ghatame- CV.pdf
College:  ['nagpur university']
---------------------------------------------- 
FileName :  Abhilash Pillai- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Abhinaw Kumar- CV.pdf
College:  ['indian institute of information technology allahabad']
---------------------------------------------- 
FileName :  Abhiram K A- CV.pdf
College:  ['osmania university']
---------------------------------------------- 
FileName :  ABHISHEK BHATIA- CV.pdf
College:  ['national institute of technology warangal']
---------------------------------------------- 
FileName :  Abhishek Shailendra Agarwal- CV.pdf
College:  ['indian institute of technology kharagpur']
---------------------------------------------- 
FileName :  Abhisheka Srivastava- CV (IKYA).pdf
College:  []
---------------------------------------------- 
FileName :  

College:  ['birla institute of technology and science pilani']
---------------------------------------------- 
FileName :  DEEPAK JAIN- CV.pdf
College:  ['st xavier s college', 'calcutta university']
---------------------------------------------- 
FileName :  Deepali Bhargava- CV.pdf
College:  ['university of delhi']
---------------------------------------------- 
FileName :  Deepesh Purswani- CV (Kelly Services).pdf
College:  ['university of lucknow']
---------------------------------------------- 
FileName :  Dhananjay Chithathoor- CV (ISHWA).pdf
College:  ['bharathiar university']
---------------------------------------------- 
FileName :  Dhiraj Mangaramani- CV.pdf
College:  ['amravati university']
---------------------------------------------- 
FileName :  Dilip Menda- CV (Ishwa).pdf
College:  []
---------------------------------------------- 
FileName :  Dinesh Chhabra- CV.pdf
College:  []
---------------------------------------------- 
FileName :  DIPESH MOHILE- CV.pdf
College: 

College:  []
---------------------------------------------- 
FileName :  KARAN MENON- CV (Ema Partners International).pdf
College:  ['burdwan university']
---------------------------------------------- 
FileName :  Karthik Medarmetla- CV.pdf
College:  ['indian institute of technology madras']
---------------------------------------------- 
FileName :  Karthik Natarajan- CV.pdf
College:  ['vivekananda college']
---------------------------------------------- 
FileName :  Kaustav Sawhny- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Kavindra Mishra- CV.pdf
College:  ['devi ahilya vishwavidyalaya']
---------------------------------------------- 
FileName :  Kavindra Pant- CV (EMA Partners International).pdf
College:  ['nagpur university']
---------------------------------------------- 
FileName :  KRISHMA AMIN- CV.pdf
College:  ['trinity college']
---------------------------------------------- 
FileName :  Kunal Garach- CV.pdf
College:  ['university of mum

College:  []
---------------------------------------------- 
FileName :  Prabhakar Sunder- CV.pdf
College:  ['madras university']
---------------------------------------------- 
FileName :  PRADEEP JANARDHANAN- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Pradeep Singh- CV (EMA Partners International).pdf
College:  []
---------------------------------------------- 
FileName :  Prakash Dalmia- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Prakash Sangam- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Pramod Singh- CV.pdf
College:  ['university of jammu']
---------------------------------------------- 
FileName :  Pranab Ghosal- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Pranjul Bhandari- CV.pdf
College:  ['st stephen s college']
---------------------------------------------- 
FileName :  Prasenjit Basu- CV.pdf
College:  ['st stephen s college']
-----

College:  ['dr b r ambedkar university']
---------------------------------------------- 
FileName :  Saurabh SARKAR- CV.pdf
College:  ['indian institute of technology delhi']
---------------------------------------------- 
FileName :  Sayaji Hande- CV.pdf
College:  ['shivaji university', 'indian statistical institute']
---------------------------------------------- 
FileName :  Sayan Dasgupta- CV.pdf
College:  ['indian statistical institute']
---------------------------------------------- 
FileName :  SHAILI MEHTA- CV.pdf
College:  ['trinity college']
---------------------------------------------- 
FileName :  Shaji Varghese- CV.pdf
College:  ['bharati vidyapeeth university']
---------------------------------------------- 
FileName :  SHALABH CHOWDHARY- CV (abc Consultants).pdf
College:  ['symbiosis institute of management studies']
---------------------------------------------- 
FileName :  Shardul Kadam- CV.pdf
College:  []
---------------------------------------------- 
FileName :  

College:  []
---------------------------------------------- 
FileName :  VINAYAK RAMAN SHARMA- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Vineet Gautam- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Vipin K- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Vipul Dave- CV (EMA Partners International).pdf
College:  []
---------------------------------------------- 
FileName :  VISHAL THAKUR- CV (Vito).pdf
College:  ['symbiosis international university', 'shri vaishnav institute of technology science']
---------------------------------------------- 
FileName :  Vishesh Sharma- CV.pdf
College:  []
---------------------------------------------- 
FileName :  VIVAN SINGH GILL- CV (ISHWA).pdf
College:  []
---------------------------------------------- 
FileName :  Yagnesh Sanghrajka- CV.pdf
College:  []
---------------------------------------------- 
FileName :  Yash Bansal- CV.pdf
College:  [

### PDF to DataFrame Conversion Pipeline

In [19]:
import pdf_to_html as pth
import multiprocessing as mp

In [18]:
RAW_DATA_PATH = "../2. Data/raw"
HTML_PATH = "../2. Data/html"
TXT_PATH = "../2. Data/txt"
PROT_DF_PATH =  "../2. Data/prot_df"

In [20]:
pdf_list=os.listdir(RAW_DATA_PATH)

In [28]:
new_pdf_list=[]
for i in pdf_list:
    temp=i.split('.')
    if temp[-1].lower().strip()=='pdf':
        new_pdf_list.append(i)
    else:
        print(i)

#### PDF to HTML

In [31]:
pool = mp.Pool(8)

results = pool.starmap(pth.pdf_to_html_function, [(raw_prot, RAW_DATA_PATH,HTML_PATH) for raw_prot in new_pdf_list])

pool.close()

#### HTML TO TEXT

In [43]:
pool = mp.Pool(8)

results = pool.starmap(pth.html_to_text, [(raw_prot,HTML_PATH,TXT_PATH) for raw_prot in new_pdf_list])

pool.close()

#### TEXT TO PROTOCOL DATFRAME

In [47]:
pool = mp.Pool(8)

results = pool.starmap(pth.text_to_dataframe, [(raw_prot,TXT_PATH,PROT_DF_PATH) for raw_prot in new_pdf_list])

pool.close()

### Other

In [32]:
import spacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

nlp = spacy.load('en_core_web_sm')

data_dir = r'C:\Users\admin\OneDrive - True North Managers LLP\Desktop\TrueNorth\1. Projects\2. CV Extraction\1. Shared\CVs'

# input_text = convertPDFToText_String(os.path.join(data_dir, os.listdir(data_dir)[0]))
input_text = convertPDFToText_String(r"C:\Users\admin\OneDrive - True North Managers LLP\Desktop\Personal\Jitendra_Jangid_Resume.pdf")

os.listdir(data_dir)[0]

file = "../2. Data/skills.txt"
file = open(file, "r", encoding='utf-8')

skill = [line.strip().lower() for line in file]
skillsmatcher = PhraseMatcher(nlp.vocab)

skill

patterns = [nlp.make_doc(text) for text in skill if len(nlp.make_doc(text)) < 10]

skillsmatcher.add("SKILLS", None, *patterns)

out_skills = []

__nlp = nlp(input_text.lower())
# Only run nlp.make_doc to speed things up

matches = skillsmatcher(__nlp)
for match_id, start, end in matches:
    span = __nlp[start:end]
    out_skills.append(span.text)

out_skills = list(set(out_skills))

out_skills

## Keyword Search Utility 
from flashtext import KeywordProcessor
import re
import string

def clean_text_func(text):
    text = str(text)
    text = text.lower()
    text = ''.join([i for i in text if not i.isdigit()])
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = text.replace("\n", " ")
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    text = translator.sub(' ', text)
    text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    text = text.strip()
    return text

class basic_keyword_extract:
    """Basic keyword extractor class using the keyword processor by Flashtext"""
    def __init__(self, keyword_list, case_sensitive=False):
        self.keyword_list = keyword_list
        self.case_sensitive = case_sensitive
        self.keyword_processor = self._initialize_basic_keyword_processor()
    def _initialize_basic_keyword_processor(self):
        kp = KeywordProcessor(case_sensitive=self.case_sensitive)
        kp.add_keywords_from_list(self.keyword_list)
        return kp
    def extract_from_text(self, raw_text):
        if self.case_sensitive:
            text = raw_text
        else:
            text = raw_text.lower().strip()
            clean_text = clean_text_func(text)
            
        clean_extracted_keywords_ls = self.keyword_processor.extract_keywords(clean_text, span_info=True)
        output_ls = list(set([i[0] for i in clean_extracted_keywords_ls]))
        if len(output_ls) == 0:
            return []
        else:
            return output_ls

## Initialize Keyword Processor:
skills_keywords_search = basic_keyword_extract(skill, case_sensitive=False)

skills_keywords_search.extract_from_text(input_text)

In [14]:
from pyresparser import ResumeParser
data_1 = ResumeParser(r"C:\Users\admin\OneDrive - True North Managers LLP\Desktop\Personal\Jitendra_Jangid_Resume (Data Scientist).pdf").get_extracted_data()
data_1

C:\Users\admin\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


{'name': 'Skills Data',
 'email': 'jitujangid38@gmail.com',
 'mobile_number': '9933915716',
 'skills': ['Gensim',
  'Research',
  'Analytics',
  'Process',
  'Ai',
  'Seaborn',
  'Matlab',
  'Writing',
  'Algorithms',
  'Forecasts',
  'Banking',
  'Forecasting',
  'Spacy',
  'Pr',
  'Design',
  'Modeling',
  'Implicit',
  'Profiling',
  'Finance',
  'Flask',
  'Pandas',
  'Octave',
  'R',
  'Jupyter',
  'Tableau',
  'Kpis',
  'Interactive',
  'Engineering',
  'Statistics',
  'Pattern',
  'Github',
  'Xgboost',
  'Python',
  'International',
  'Data analytics',
  'Android',
  'Spyder',
  'Visual',
  'Coding',
  'Jira',
  'Nltk',
  'C',
  'Matplotlib',
  'Strategy',
  'Tensorflow',
  'Analysis',
  'Numpy',
  'System',
  'Parser'],
 'college_name': None,
 'degree': ['B.TECH'],
 'designation': ['DATA SCiENCE ASSOCiATE CONSULTANT'],
 'experience': ['True North Co',
  'DATA SCiENTiST',
  'ZS Associates',
  'DATA SCiENCE ASSOCiATE CONSULTANT',
  'Jitendra Jangid',
  'DATA SCiENTiST',
  '542 G

In [16]:
from resume_parser import resumeparse

data_2 = resumeparse.read_file(r"C:\Users\admin\OneDrive - True North Managers LLP\Desktop\Personal\Jitendra_Jangid_Resume (Data Scientist).pdf")
data_2

C:\Users\admin\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


{'email': 'jitujangid38@gmail.com',
 'phone': '(+91) 9933915716',
 'name': 'Jitendra Jangid',
 'total_exp': 4,
 'university': ['indian institute of technology, kharagpur'],
 'designition': ['data scientist',
  'scientist',
  'ceo',
  'cfo',
  'associate',
  'student'],
 'degree': [],
 'skills': ['Data Science Linear/Logistic Regression',
  ' SVM',
  ' Naive Bayes',
  ' K‑Means',
  ' Random Forest',
  ' GBM',
  ' XGBoost',
  ' LightGBM',
  ' CatBoost',
  ' CNN',
  ' RNN',
  ' LSTMs',
  'Libraries Scikit‑learn',
  ' Numpy',
  ' Pandas',
  ' Matplotlib',
  ' Seaborn',
  ' Plotly',
  ' TensorFlow',
  ' Gensim',
  ' Pattern',
  ' NLTK',
  ' SpaCy',
  ' StanfordCoreNLP Python',
  ' R',
  ' C',
  ' Matlab',
  ' Octave Amazon Sagemaker',
  ' Databricks',
  ' Jupyter Notebook/Jupyterlab',
  ' Spyder',
  ' Visual Studio',
  ' Github/BitBucket',
  ' Jira',
  ' Tableau'],
 'Companies worked at': ['Cab Company Pvt. Ltd.',
  'https://github.com/overfitter https://www.linkedin.com/in/jitendraiitkgp01

In [ ]:
for file in os.listdir(directory):
    try:
        if not file.endswith(".pdf"):
            continue
        out_dict = resumeparse.read_file(os.path.join(data_dir, file))
        print("FileName : ",file)
        print("University :", ", ".join(set(out_dict["university"])))
        print("Degree :", ", ".join(set(out_dict["degree"])))
        skills = [i.strip() for i in out_dict["skills"]]
        print("Skills : ", ", ".join(set(skills)),end="\n---------------------------------------------- \n")
    except Exception as e:
        print("Error: ", file)
        continue

In [ ]:
for file in os.listdir(PROT_DF_PATH):
    print("FileName : ",file.replace("prot_df_","").replace(".csv",".pdf").strip())
    df = pd.read_csv(os.path.join(PROT_DF_PATH, file))
    text = " ".join(df[(df["tag_type"]=="Data") & (df["is_bold"]==True)]["tag_value"].tolist())
    text = clean_special_text(text)
    company = list(set(company_keywords_search.extract_from_text(text)))
    print("Companies: ", company, end="\n---------------------------------------------- \n")